# OAC-Supported Publications
From a provided list of grants, extract and rank publications by various metrics built into Dimensions

In [1]:
!pip install dimcli -U --quiet

import dimcli
from dimcli.utils import *
import os,sys,time,json
from os.path import exists
import copy
import pandas as pd
import more_itertools

print ("Logging in")
#Use dsl.ini file
dsl=dimcli.login()
dsl=dimcli.Dsl()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.5 MB/s eta 0:00:00


Searching config file credentials for default 'live' instance..


Logging in
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://nsf.dimensions.ai/api/dsl> - DSL v2.7
Method: dsl.ini file


In [2]:
#Upload the grant information file if running in Colab.
grants_source = pd.read_excel('SSI2011.xlsx',verbose=False)
# Uncomment this to inspect
#grants_source.head(3)
print("Number of awards", len(grants_source))

Number of awards 77


In [3]:
# Send this function an NSF grant number and it returns the internal Dimensions grant ID.
def findDimensionGrantID(nsfGrantNumber):
  grant = dsl.query(f'''extract_grants(grant_number="{nsfGrantNumber}", funder_name="National Science Foundation")''',verbose=False).json
  grant_id = grant.get("grant_id")
  return grant_id

In [4]:
# Returns all the publications that Dimensions associates with the input grant_id.
def getPublicationsForGrant(grantID):
  queryTemplate="""search publications
    where supporting_grant_ids={}
    return publications[id+doi+times_cited+supporting_grant_ids+field_citation_ratio+altmetric+relative_citation_ratio]
    limit 1000"""
  data = dsl.query(queryTemplate.format(grantID), verbose=False).as_dataframe()
  return data

In [5]:
awardNumberSeries=grants_source[0]

#Get the grantID and associated publications
grant_pubs_df=pd.DataFrame()
for x in awardNumberSeries:
  grantID=findDimensionGrantID(int(x))
  #print(grantID)
  #grant_pubs_df=grant_pubs_df.append(getPublicationsForGrant(json.dumps(grantID)))
  grant_pubs_df=pd.concat([grant_pubs_df,getPublicationsForGrant(json.dumps(grantID))])

grant_pubs_df


Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.
Too Many Requests for the Server. Sleeping for 30 seconds and then retrying.


,id,altmetric,doi,field_citation_ratio,supporting_grant_ids,times_cited,relative_citation_ratio
0,pub.1017709537,3.0,10.1103/physreva.90.052501,2.29,[grant.3132622],9,NaN
1,pub.1051091210,NaN,10.1103/physrevb.86.125310,3.72,[grant.3132622],15,NaN
0,pub.1071474954,36.0,10.3847/1538-4357/834/2/137,6.89,"[grant.3934096, grant.3124939, grant.3125047, ...",24,NaN
1,pub.1071474889,74.0,10.3847/1538-4357/834/1/72,12.93,"[grant.3934096, grant.3000498, grant.3125047, ...",45,NaN
2,pub.1071474009,98.0,10.3847/0004-637x/831/2/150,21.35,"[grant.3124873, grant.3000498, grant.3132624, ...",82,NaN
...,...,...,...,...,...,...,...
42,pub.1060644363,NaN,10.1103/physrevb.90.054102,2.77,"[grant.3004453, grant.3479298]",30,NaN
43,pub.1000663803,1.0,10.1088/1367-2630/16/1/015018,NaN,"[grant.3004453, grant.2995536]",68,NaN
0,pub.1093055870,128.0,10.1186/s12859-017-1934-z,1425.02,"[grant.3637234, grant.2669336, grant.3129854]",4108,121.53
1,pub.1053263378,1.0,10.1186/s12859-016-1383-0,17.66,[grant.3129854],26,0.56


In [ ]:
grant_pubs_df.to_excel("SSI2011GrantPubsRankings.xlsx", index=False)